In [5]:
import pandas as pd

# Binary or 3 label prediction?

Choose the label correctly


In [6]:
if_binary = 0

Compile the dataset now

In [9]:
df = pd.read_csv(
    "/Users/akseljoonas/Documents/news-sentiment/data/processed/news+stocks.csv"
)

In [ ]:
df

In [ ]:
df['ticker'].value_counts()


In [ ]:
# Check how many cases have the same values in 'summary' and 'description'
matching_cases_count = (df["summary"] == df["description"]).sum()

# Output the result
print(
    f"Number of cases where 'summary' and 'description' are the same: {matching_cases_count} of lenght {len(df)}"
)

In [5]:
# 1. Merge the text in 'title', 'summary', and 'description' into a new column 'text'
df["text"] = (
    df["title"].fillna("")
    + " "
    + df["summary"].fillna("")
)


In [6]:
# 2. Remove '<>:/\|?*' characters from the 'text' column
df["text"] = df["text"].str.replace(r"[<>:/\\|?*]", "", regex=True)


In [7]:
df.rename(columns={"increased_3_percent": "labels"}, inplace=True)

In [8]:
if not if_binary:

    def label_price_move(row):
        if row < -0.03:
            return 0
        elif row > 0.03:
            return 2
        else:
            return 1


    # Apply the labeling function to the price_move column
    df["labels"] = df["price_move"].apply(label_price_move)

In [9]:
df = df[["text", 'topic' ,"labels"]]

In [10]:
df.drop_duplicates(inplace=True)

In [ ]:
df_relevant = df[df["labels"].isin([0, 2])]
df_irrelevant = df[df["labels"].isin([1])]
result = (
    df_relevant.groupby("topic")["labels"]
    .count()
    .reset_index(name="relevant_label_count")
)
result1 = (
    df_irrelevant.groupby("topic")["labels"]
    .count()
    .reset_index(name="irrelevant_label_count")
)
result = pd.merge(result, result1, on='topic', how="outer")

result_sorted = result.sort_values(by="relevant_label_count", ascending=False)
result_sorted

In [76]:
min_label_count = 5

excluded_topics = [
    "calendar_of_events",
    "conference_calls__webcasts",
    "directors_and_officers",
    "company_announcements",
    "european_regulatory_news",
]


# Filter topics with relevant_label_count >= 7
filtered_topics = result_sorted[
    result_sorted["relevant_label_count"] >= min_label_count
]["topic"].tolist()

# Remove excluded topics from the filtered topics
filtered_topics = [topic for topic in filtered_topics if topic not in excluded_topics]

# Now filter the original DataFrame based on the remaining filtered topics
df = df[df["topic"].isin(filtered_topics)]

# df.drop('topic',inplace=True, axis=1)
# Optionally, reset the index for cleanliness
df.reset_index(drop=True, inplace=True)

In [ ]:
df['labels'].value_counts()

In [12]:
df.drop('topic',inplace=True, axis=1)

In [13]:
# 4. Save the modified DataFrame to a new CSV file
df.to_csv(
    "./datasets_big/dataset_finetuning_3_labels.csv",
    index=False,
)